In [4]:
import import_ipynb
from Reservoir import Reservoir

import os
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import tqdm_notebook
from tensorboardX import SummaryWriter

# Ignore warnings
import warnings
import time
warnings.filterwarnings("ignore")

os.chdir("./MNIST")
from MNIST_model import ConvNet
os.chdir("./..")

In [5]:
class Model():
    
    def __init__(self,device):
        outputs = 10
        image_shape = (1,28,28)
        self.device = device
        print("start network")
        self.network = ConvNet(outputs, image_shape).to(device)
        print("done network")
        self.optimizer = optim.Adam(self.network.parameters(), amsgrad=True, weight_decay=0.01)
        self.criterion = nn.CrossEntropyLoss()
        self.start_epoch = 0
        print("done init")
        
    def load_weights(self, load_path):
        checkpoint = torch.load(load_path)
        self.network.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.start_epoch = checkpoint['epoch']
        
    def save_weights(self,epoch, save_path):
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.network.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()}, 
            save_path)
       
    def current_snapshot_name(self):
        from time import gmtime, strftime
        import socket

        hostname = socket.gethostname()

        date = strftime("%b%d_", gmtime())
        clock = strftime("%X", gmtime())
        now = clock.split(":")
        now = date+'-'.join(now)

        name = now+"_"+hostname
        return name

In [6]:
if __name__ == "__main__":
    dataset = Reservoir("./data/training/images/train-images-idx3-ubyte.gz",
                        "./data/training/labels/train-labels-idx1-ubyte.gz",
                        "./data/testing/images/t10k-images-idx3-ubyte.gz",
                        "./data/testing/labels/t10k-labels-idx1-ubyte.gz")
    dataloader = dataset.training_pool.dataloader
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Model(device)
    for i, batch in enumerate(dataloader):
        if i > 0:
            break
        imgs = batch['image'].float()
        print(imgs[0].shape)
        print("input", imgs.shape)
        out = model.network(imgs)
        print("output", out)

Total training stacks 5400
Total validation stacks 600
Total testing stacks 10000
start network
units after conv 588
fc parameters:  5890


KeyboardInterrupt: 